In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.signal import find_peaks
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Load data tables
Refer = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/RiverReferenceTable.csv')
Discharge = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/Ingestion/CurrentWater.csv')
Precip = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/Ingestion/CurrentPrecip.csv')
forecast_precip = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/Ingestion/NewWeatherForecast.csv')
# Standardize Date and River Name
Discharge['Date'] = pd.to_datetime(Discharge.DateTime).dt.date
Discharge['CFS'] = Discharge['Value']

####Discharge
#Join river data with reference data
Discharge = Discharge.set_index('Name').join(Refer.set_index('USGS Name')).reset_index()[[
    'Date','CFS','Name','RiverName','FishType']]

# Create Avg Daily Discharge
Avg_Discharge = Discharge.groupby(['Date','Name','RiverName','FishType']).mean().reset_index()
Avg_Discharge['Date_Name'] = Avg_Discharge.Date.astype('str')+'_'+Avg_Discharge.Name
# Remove incorrect data
Avg_Discharge = Avg_Discharge.loc[Avg_Discharge.CFS>0]

####Precipitiation
# Standardize Date
Precip['Date'] = pd.to_datetime(Precip.Date).dt.date

#Join river data with reference data
Precip = Precip.set_index('Name').join(Refer.set_index('Name')).reset_index()[[
    'Date','Precip','Name','RiverName','FishType']]

# Create Primary Key
Precip['Date_Name'] = Precip.Date.astype('str')+'_'+Precip.Name

# Create Avg Daily Precip
Precip = Precip[['Date_Name','Date','Name','Precip']].groupby(['Date','Name','Date_Name']).mean().reset_index()

#Join river data with precip data
Rivers_Flow_Rain = Avg_Discharge.set_index('Date_Name').join(Precip[['Date_Name','Precip']].set_index('Date_Name'))

#Fill precip nulls with 0
Rivers_Flow_Rain['Precip'] = Rivers_Flow_Rain.Precip.fillna(0)

# Rename completed dataframe
full = Rivers_Flow_Rain.loc[Rivers_Flow_Rain.Precip.isnull()==False]

Rivers = []
for river in full.Name.unique():
    df = full.loc[full.Name == river]
    df.reset_index(inplace=True)
    # df = df.drop(columns='index')

    # Create column for slope of discharge
    df['CFS_Slope'] = df.CFS.diff()

    # Retrieve values of Slope
    SlopeValues = df.CFS_Slope.values

    # Define the threshold for what is a good day for fishing
    #       - slope between day CFS values must be at least 1 STD from the mean 
    border = df.CFS_Slope.mean()+(df.CFS_Slope.std()/2)

    # Define troughs that are below the threshold
    peaks, _ = find_peaks(-SlopeValues, height=border)

    # Logic to build the best day flag
    #       - if the value is a trough point or has a negative low and is above 1 STD from the mean then also consider a good day 
    bestdays = []
    for i in range(len(df.CFS)):
        if i in peaks:
            x = 1
        elif (df.CFS_Slope[i]<0) & (df.CFS[i]>=border):
            x = 1
        else: x = 0
        bestdays.append(x)
    df['BestDays'] = bestdays
    Rivers.append(df)

Rivers_Flow = pd.concat(Rivers)

Rivers_Flow.head()

<ipython-input-1-2ea2f28048c5>:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CFS_Slope'] = df.CFS.diff()
<ipython-input-1-2ea2f28048c5>:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['BestDays'] = bestdays


,Date_Name,Date,Name,RiverName,FishType,CFS,Precip,CFS_Slope,BestDays
0,2021-12-04_East Fork Carson,2021-12-04,East Fork Carson,East Fork,Trout,85.455000,0.000000,NaN,0
1,2021-12-05_East Fork Carson,2021-12-05,East Fork Carson,East Fork,Trout,82.156250,0.000000,-3.298750,0
2,2021-12-06_East Fork Carson,2021-12-06,East Fork Carson,East Fork,Trout,85.530208,3.800000,3.373958,0
3,2021-12-07_East Fork Carson,2021-12-07,East Fork Carson,East Fork,Trout,91.935069,1.250000,6.404861,0
4,2021-12-08_East Fork Carson,2021-12-08,East Fork Carson,East Fork,Trout,85.727431,2.533333,-6.207639,0


In [13]:
#WindDirection - Daily Avg WindDirection 
directions = {'E':{'N':0,'E':1,'S':0,'W':0}
              ,'SW':{'N':0,'E':0,'S':.5,'W':.5}
              ,'SSE':{'N':0,'E':.25,'S':.75,'W':0}
              ,'ENE':{'N':.25,'E':.75,'S':0,'W':0}
              ,'S':{'N':0,'E':0,'S':1,'W':0}
              ,'ESE':{'N':0,'E':.75,'S':.25,'W':0}
              ,'SE':{'N':0,'E':.5,'S':.5,'W':0}
              ,'N':{'N':1,'E':0,'S':0,'W':0}
              ,'NNW':{'N':.75,'E':0,'S':0,'W':.25}
              ,'NE':{'N':.5,'E':.5,'S':0,'W':0}
              ,'NNE':{'N':.75,'E':.25,'S':0,'W':0}
              ,'NW':{'N':.5,'E':0,'S':0,'W':.5}
              ,'SSW':{'N':0,'E':0,'S':.75,'W':.25}
              ,'WSW':{'N':0,'E':0,'S':.25,'W':.75}
              ,'WNW':{'N':.25,'E':0,'S':0,'W':.75}
              ,'W':{'N':0,'E':0,'S':0,'W':1}}

forecast_precip[['WindNorth','WindEast','WindSouth','WindWest']] = forecast_precip.windDirection.apply(lambda x: pd.Series({'WindNorth':directions[x]['N']
                                                                                                ,'WindEast':directions[x]['E']
                                                                                                ,'WindSouth':directions[x]['S']
                                                                                                ,'WindWest':directions[x]['W']}))

days = {1:0,2:0,3:1,4:1,5:2,6:2,7:3,8:3,9:4,10:4,11:5,12:5,13:6,14:6}

forecast_precip['DaysFromPresent'] = forecast_precip.ForecastFromPresent.apply(lambda x: days[x])
forecast_precip['Forecast_Date_Name'] = forecast_precip.endTime.astype(str)+'_'+forecast_precip.StationName
forecast_precip['Forecast_Date_Name_Days_From_Present'] = forecast_precip.endTime.astype(str)+'_'+forecast_precip.StationName+'_'+forecast_precip.DaysFromPresent.astype(str)
forecast_precip.loc[(forecast_precip.StationName=='Eel River Fort Seward') & (forecast_precip.DaysFromPresent==0)].sort_values('Date_Name').head()
#forecast_precip.reset_index(inplace=True)

#Rivers_Flow.set_index('Date_Name').join(forecast_precip.set_index('Forecast_Date_Name')[['Forecast']])

,Unnamed: 0,Date_Name,ForecastFromPresent,StationName,WeekdayName,startTime,endTime,windDirection,Rain,WindMph,temperature,SnowFlag,WindNorth,WindEast,WindSouth,WindWest,DaysFromPresent,Forecast_Date_Name,Forecast_Date_Name_Days_From_Present
2,2,2022-12-06_Eel River Fort Seward,1,Eel River Fort Seward,Today,2022-12-06,2022-12-06,E,0.0,2.500000,41.000000,0.0,0.0,1.00,0.00,0.0,0,2022-12-06_Eel River Fort Seward,2022-12-06_Eel River Fort Seward_0
422,422,2022-12-07_Eel River Fort Seward,1,Eel River Fort Seward,Today,2022-12-07,2022-12-07,ESE,0.5,2.000000,39.000000,0.0,0.0,0.75,0.25,0.0,0,2022-12-07_Eel River Fort Seward,2022-12-07_Eel River Fort Seward_0
623,623,2022-12-08_Eel River Fort Seward,1,Eel River Fort Seward,Thursday,2022-12-08,2022-12-08,SE,2.0,9.500000,38.000000,0.0,0.0,0.50,0.50,0.0,0,2022-12-08_Eel River Fort Seward,2022-12-08_Eel River Fort Seward_0
828,828,2023-01-03_Eel River Fort Seward,1,Eel River Fort Seward,Today,2023-01-03,2023-01-03,ESE,2.0,6.666667,41.333333,0.0,0.0,0.75,0.25,0.0,0,2023-01-03_Eel River Fort Seward,2023-01-03_Eel River Fort Seward_0
1053,1053,2023-01-16_Eel River Fort Seward,1,Eel River Fort Seward,M.L. King Jr. Day,2023-01-16,2023-01-16,ESE,2.0,5.333333,42.000000,0.0,0.0,0.75,0.25,0.0,0,2023-01-16_Eel River Fort Seward,2023-01-16_Eel River Fort Seward_0


In [89]:
fore_eel = forecast_precip.loc[(forecast_precip.StationName=='Eel River Fort Seward')]
fore_eel  = fore_eel.pivot_table(values='Rain',index='Forecast_Date_Name',columns='DaysFromPresent').tail(25)

# If NaN, take the next row's value. If the next row value is NaN, then take the next column's value.
row = 0
col = 0
#fore_eel.iloc[row][0]

# for every row in df
while row<len(fore_eel[0])-1:

    # look at each column's value
    while col<7:
        
        # if it's NaN
        if fore_eel.iloc[row][col] == np.nan:

            
            # then the value = next row's value
            value = fore_eel.iloc[row+1][col]

        # if the next row's value is also NaN
        elif fore_eel.iloc[row+1][col] == np.nan:
                
                # then the value = the next column's value
                value = fore_eel[row][col+1]
        
        # otherwise the value = current value
        elif fore_eel.iloc[row][col] != np.nan:
             value = fore_eel.iloc[row][col]
        
        else: value = 0
        print(row,col,value)
        
        # look at the next column's value
        col=col+1 
    col = 0 
    row=row+1
    
         

0 0 0.0
0 1 nan
0 2 nan
0 3 0.0
0 4 nan
0 5 0.5
0 6 0.0
1 0 0.0
1 1 0.0
1 2 nan
1 3 nan
1 4 0.5
1 5 nan
1 6 0.5
2 0 0.0
2 1 0.0
2 2 0.0
2 3 nan
2 4 nan
2 5 0.0
2 6 nan
3 0 1.0
3 1 1.0
3 2 2.0
3 3 2.0
3 4 nan
3 5 nan
3 6 0.0
4 0 1.0
4 1 1.0
4 2 0.0
4 3 0.0
4 4 0.0
4 5 nan
4 6 nan
5 0 nan
5 1 0.0
5 2 0.0
5 3 0.0
5 4 0.0
5 5 0.0
5 6 nan
6 0 nan
6 1 nan
6 2 0.0
6 3 0.0
6 4 0.0
6 5 0.0
6 6 0.0
7 0 0.0
7 1 nan
7 2 nan
7 3 1.0
7 4 0.5
7 5 1.0
7 6 1.0
8 0 0.0
8 1 0.0
8 2 nan
8 3 nan
8 4 0.0
8 5 0.0
8 6 0.0
9 0 0.0
9 1 0.5
9 2 1.0
9 3 nan
9 4 nan
9 5 1.0
9 6 1.0
10 0 0.0
10 1 0.0
10 2 0.5
10 3 2.0
10 4 nan
10 5 nan
10 6 2.0
11 0 nan
11 1 0.0
11 2 0.0
11 3 0.0
11 4 0.0
11 5 nan
11 6 nan
12 0 nan
12 1 nan
12 2 0.0
12 3 0.0
12 4 0.0
12 5 0.0
12 6 nan
13 0 0.0
13 1 nan
13 2 nan
13 3 0.0
13 4 0.0
13 5 0.0
13 6 0.0
14 0 1.5
14 1 1.5
14 2 nan
14 3 nan
14 4 0.5
14 5 0.5
14 6 0.0
15 0 1.3333333333333333
15 1 1.0
15 2 1.0
15 3 nan
15 4 nan
15 5 1.5
15 6 1.0
16 0 0.0
16 1 0.0
16 2 0.0
16 3 0.0
16 4 nan
16

In [73]:
fore_eel

DaysFromPresent,0,1,2,3,4,5,6
Forecast_Date_Name,,,,,,,
2023-02-07_Eel River Fort Seward,0.000000,NaN,NaN,0.0,NaN,0.5,0.00
2023-02-08_Eel River Fort Seward,0.000000,0.0,NaN,NaN,0.5,NaN,0.50
2023-02-09_Eel River Fort Seward,0.000000,0.0,0.0,NaN,NaN,0.0,NaN
2023-02-10_Eel River Fort Seward,1.000000,1.0,2.0,2.0,NaN,NaN,0.00
2023-02-11_Eel River Fort Seward,1.000000,1.0,0.0,0.0,0.0,NaN,NaN
2023-02-12_Eel River Fort Seward,NaN,0.0,0.0,0.0,0.0,0.0,NaN
2023-02-13_Eel River Fort Seward,NaN,NaN,0.0,0.0,0.0,0.0,0.00
2023-02-14_Eel River Fort Seward,0.000000,NaN,NaN,1.0,0.5,1.0,1.00
2023-02-15_Eel River Fort Seward,0.000000,0.0,NaN,NaN,0.0,0.0,0.00


In [4]:
# Reduce data to Eel River only
new_river = Rivers_Flow.loc[Rivers_Flow['Name']=='Eel River Scotia']
new_river.reset_index(inplace=True)
new_river = new_river .drop(columns='index')
new_river['Precip_Shift'] = new_river.Precip.shift(1)

new_river['CFS_Rolling'] = new_river.CFS.rolling(window=3,min_periods=1).mean()
new_river['CFS_Slope_Rolling'] = new_river.CFS.rolling(window=3,min_periods=1).mean()
new_river['CFS_Rolling30'] = new_river.CFS.rolling(window=30,min_periods=1).mean()
new_river['new_variable'] = np.abs(new_river.CFS) - new_river.CFS_Rolling30

model = SARIMAX(new_river.CFS[1:],exog=new_river[['Precip_Shift','CFS_Rolling']].dropna(0),order=(0,1,0))
model_fit = model.fit()

length = len(new_river.CFS)
forecast = model_fit.forecast(length,exog=new_river[['Precip_Shift','CFS_Rolling']].fillna(0))

# Create figure with secondary y-axis
fig = make_subplots(rows=2,cols=1)
# Add traces
fig.add_trace(
    go.Scatter(x=new_river.Date, y=new_river.CFS, name="CFS - Observed"),
    secondary_y=False,row=1,col=1)
# fig.add_trace(
#     go.Scatter(x=new_river.Date, y=predict, name="CFS - Predicted"),
#     secondary_y=False,row=2,col=1)
fig.add_trace(
    go.Scatter(x=new_river.Date, y=forecast, name="CFS - Forecast"),
    secondary_y=False,row=2,col=1)

# Add figure title
fig.update_layout(
    title_text="Observed, Predicted, & Forecasted ")
# Set x-axis title
fig.update_xaxes(title_text="Date")
# Set y-axes titles
fig.update_yaxes(title_text="Discharge (cfs)", secondary_y=False)
fig.update_yaxes(title_text="Precip (mm)", secondary_y=True)
fig.show()

c:\Users\Scott\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:590: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
c:\Users\Scott\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:590: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
c:\Users\Scott\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:390: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'
